In [ ]:
%pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
! pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-keaxolxn
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-keaxolxn
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=73438aca73207db849588d4c1d14ac117ca3c41067193b7dc6d64a71a30bd11e
  Stored in directory: /tmp/pip-ephem-wheel-cache-t19cdpny/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import keras

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import tensorflow_addons
from keras_contrib.layers import CRF

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
#Reading the csv file
df = pd.read_csv('/content/demo_ner2.csv')

In [ ]:
X_data = df.Sentence
y_data = df.Label

In [ ]:
y_data

0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, ...
1        [1, 4, 4, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0]
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, ...
3        [1, 4, 4, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0]
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, ...
5        [1, 4, 4, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0]
6    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, ...
7        [1, 4, 4, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0]
8    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, ...
9        [1, 4, 4, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0]
Name: Label, dtype: object

In [ ]:
X_data[0]

'Tuy_nhiên , điểm đáng_giá là chi_tiết được tái_tạo rất chân_thực , không bị gai_góc như_trên Samsung .'

In [ ]:
# Define a function to extract words and punctuation from a sentence
def extract_words_and_punctuation(sentence):
    words_and_punctuation = re.findall(r'\w+|[.,!?;]', sentence.lower())  # Split on words and punctuation marks
    return words_and_punctuation

# Apply the function to each row of the "Sentence" column and store the results in a list
unique_words_and_punctuation = set()
for sentence in df['Sentence']:
    words_and_punctuation = extract_words_and_punctuation(sentence)
    unique_words_and_punctuation.update(words_and_punctuation)

# Convert the set of unique words and punctuation marks back to a list
unique_words_list = list(unique_words_and_punctuation)

word_to_index = {w : i + 2 for i, w in enumerate(unique_words_list)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0
idx2word = {i: w for w, i in word_to_index.items()}

In [ ]:
idx2word

{2: 'như_trên',
 3: 'tuy_nhiên',
 4: 'thông_số',
 5: 'điểm',
 6: 'cho_đến',
 7: 'gai_góc',
 8: 'không',
 9: 'được',
 10: 'ip68',
 11: '.',
 12: 'tốt',
 13: 'cả_hai',
 14: 'chân_thực',
 15: 'đều',
 16: 'sở_hữu',
 17: 'samsung',
 18: 'cao_cấp',
 19: ',',
 20: 'là',
 21: 'cường_lực',
 22: 'bảo_vệ',
 23: 'bị',
 24: 'kính',
 25: 'từ',
 26: 'tái_tạo',
 27: 'rất',
 28: 'những',
 29: 'chi_tiết',
 30: 'đáng_giá',
 1: 'UNK',
 0: 'PAD'}

In [ ]:
# Define a function to convert a sentence to a list of indexes
def sentence_to_indexes(sentence, word_to_index):
    words = sentence.lower().split()  # Split sentence into words and convert to lowercase
    indexes = [word_to_index.get(word, word_to_index["UNK"]) for word in words]
    return indexes

# Apply the function to each element in the array X and store the results in a new list
X_indexed = np.array([sentence_to_indexes(sentence, word_to_index) for sentence in X_data])

<ipython-input-93-75d574d446e0>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_indexed = np.array([sentence_to_indexes(sentence, word_to_index) for sentence in X_data])


In [ ]:
# Padding each sequence to have same length  of each word
X_pad = pad_sequences(maxlen = 128, sequences = X_indexed, padding = "post", value = word_to_index["PAD"])

In [ ]:
X_pad[0]

array([ 3, 19,  5, 30, 20, 29,  9, 26, 27, 14, 19,  8, 23,  7,  2, 17, 11,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [ ]:
import ast
y_indexed = [ast.literal_eval(y_str) for y_str in y_data]
y_indexed = np.array([np.array(lst) for lst in y_indexed])

<ipython-input-17-389b7664c8fa>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_indexed = np.array([np.array(lst) for lst in y_indexed])


In [ ]:
y_pad = pad_sequences(maxlen = 128, sequences = y_indexed, padding = "post", value = word_to_index["PAD"])

In [ ]:
y_pad[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
y_categorical = np.array([to_categorical(i, num_classes = 5) for i in y_pad])

In [ ]:
y_categorical[0]

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_categorical, test_size = 0.15)

In [ ]:
y_train.shape

(10, 128, 5)

In [ ]:
X_train.shape

(10, 128)

In [ ]:
# num_tags = df['Tag'].nunique()
# Model architecture
embedding = 128
max_len = 128
input = tf.keras.Input(shape = (max_len,))
model = tf.keras.layers.Embedding(input_dim = len(unique_words_list) + 2, output_dim = embedding, input_length = max_len)(input)
model = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 64, return_sequences=True, recurrent_dropout=0.1))(model)
# model = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(100, activation="relu"))(model)
model = tf.keras.layers.Dense(128, activation='relu')(model)
#crf = CRF(5)
#out = crf(model)
out = tf.keras.layers.Dense(5, activation='softmax')(model)  # output

model = tf.keras.Model(input, out)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_7 (Embedding)     (None, 128, 128)          3968      
                                                                 
 bidirectional_7 (Bidirecti  (None, 128, 128)          98816     
 onal)                                                           
                                                                 
 dense_12 (Dense)            (None, 128, 128)          16512     
                                                                 
 dense_13 (Dense)            (None, 128, 5)            645       
                                                                 
Total params: 119941 (468.52 KB)
Trainable params: 119941 (468.52 KB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
batch_size = 64
epochs = 8
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/8
1/1 [==============================] - 8s 8s/step - loss: 1.6799 - accuracy: 0.0063
Epoch 2/8
1/1 [==============================] - 1s 863ms/step - loss: 1.6178 - accuracy: 0.0523
Epoch 3/8
1/1 [==============================] - 1s 881ms/step - loss: 1.5671 - accuracy: 0.9547
Epoch 4/8
1/1 [==============================] - 1s 888ms/step - loss: 1.5281 - accuracy: 0.9547
Epoch 5/8
1/1 [==============================] - 2s 2s/step - loss: 1.4855 - accuracy: 0.9547
Epoch 6/8
1/1 [==============================] - 2s 2s/step - loss: 1.4355 - accuracy: 0.9547
Epoch 7/8
1/1 [==============================] - 1s 895ms/step - loss: 1.3751 - accuracy: 0.9547
Epoch 8/8
1/1 [==============================] - 1s 962ms/step - loss: 1.3010 - accuracy: 0.9547


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

1/1 [==============================] - 1s 1s/step - loss: 1.2774 - accuracy: 0.9453
Test Loss: 1.2774338722229004
Test Accuracy: 0.9453125


In [ ]:
test_x_text = "chi_tiết đáng_giá từ adsadsadada"
test_x = np.array([sentence_to_indexes(test_x_text, word_to_index)])
test_x_pad = pad_sequences(maxlen = 128, sequences = test_x, padding = "post", value = word_to_index["PAD"])

In [ ]:
test_x_pad

array([[29, 30, 25,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
      dtype=int32)

In [ ]:
model.predict(test_x_pad)

1/1 [==============================] - 0s 179ms/step


array([[[0.23646177, 0.20616342, 0.1896774 , 0.19182476, 0.17587258],
        [0.24170369, 0.20678662, 0.18753983, 0.19104694, 0.17292298],
        [0.24533474, 0.20743977, 0.1874896 , 0.19052814, 0.16920777],
        [0.24907276, 0.20907164, 0.18972582, 0.18639438, 0.16573535],
        [0.25915286, 0.20798592, 0.18781537, 0.18568207, 0.15936375],
        [0.26302603, 0.20823044, 0.18657848, 0.18391576, 0.15824926],
        [0.26632786, 0.2082697 , 0.18570124, 0.18207824, 0.15762307],
        [0.2694714 , 0.20816454, 0.18492945, 0.18028307, 0.15715156],
        [0.27257276, 0.208116  , 0.18430354, 0.17841718, 0.15659052],
        [0.27536625, 0.20808777, 0.18366344, 0.17681083, 0.1560717 ],
        [0.27768642, 0.2080595 , 0.183149  , 0.17545292, 0.15565223],
        [0.27967486, 0.2079738 , 0.18273918, 0.17427948, 0.15533267],
        [0.28134614, 0.20775242, 0.18248592, 0.17335285, 0.15506266],
        [0.28280547, 0.2076624 , 0.18219477, 0.17254849, 0.15478884],
        [0.28407374,